[Reference](https://medium.com/@ricardomoura_90507/developing-an-fastapi-with-python-and-docker-ca6e80021559)

In [4]:
# # requirements.txt
# pylint==2.13.5
# pytest==7.1.1
# streamlit==1.8.1
# fastapi==0.68.0
# pydantic==1.8.0
# uvicorn==0.15.0

In [5]:
'''
This file contains the FastAPI application that defines the /items/ route for creating items. The Item class is defined using Pydantic for data validation. The /items/ route expects to receive an instance of Item
as input and returns a dictionary containing the item's information, as well as the price with tax (if the tax is defined).
'''

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    description: str = None
    price: float
    tax: float = None

@app.post("/items/")
async def create_item(item: Item):
    '''
    Creates a new item based on the data sent in the request.
    Args:
    item (Item): Data of the item sent in the request.
    Returns:
    dict: Dictionary containing the item's information, as well as the price with tax (if the tax is defined).
    '''
    item_dict = item.dict()
    if item.tax:
        price_with_tax = item.price + item.tax
        item_dict.update({"price_with_tax": price_with_tax})
    return item_dict

In [6]:
import requests
import streamlit as st
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    description: str = None
    price: float
    tax: float = None

def create_item(item: Item):
    '''
    Create a new item by sending a POST request to the FastAPI API.
    Args:
    item (Item): Data of the item to be created.
    Returns:
    dict: Dictionary containing the information of the created item.
    '''
    item_dict = item.dict()
    response = requests.post("http://localhost/items/", json=item_dict)
    return response.json()

st.title("Create an item")
name = st.text_input("Name")
description = st.text_input("Description")
price = st.number_input("Price", step=0.01)
tax = st.number_input("Tax", step=0.01)
item = Item(name=name, description=description, price=price, tax=tax)

if st.button("Create"):
  # Create the item and display the response.
  response = create_item(item)
  st.write(response)

In [7]:
# Dockerfile
# Start from the official Python base image.
FROM python:3.9.11-buster

# Set the current working directory to /project.
# This is where we'll put the requirements.txt file and the app directory.
WORKDIR /project

# Copy the file with the requirements to the /project directory.
# Copy only the file with the requirements first, not the rest of the code.
# As this file doesn't change often, Docker will detect it and use the cache for this step, enabling the cache for the next step too.
COPY ./requirements.txt /project/requirements.txt

# Install the package dependencies in the requirements file.
# The --no-cache-dir option tells pip to not save the downloaded packages locally, as that is only if pip was going to be run again to install the same packages, but that's not the case when working with containers.
# The --upgrade option tells pip to upgrade the packages if they are already installed.
RUN pip install --no-cache-dir --upgrade -r /project/requirements.txt

# Copy the ./app directory inside the /project directory.
COPY ./app /project/app

# Set the command to run the uvicorn server.
# CMD takes a list of strings, each of these strings is what you would type in the command line separated by spaces.
# This command will be run from the current working directory, the same /project directory you set above with WORKDIR /project.
# Because the program will be started at /project and inside of it is the directory ./app with your code, Uvicorn will be able to see and import app from app.main.
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "80"]